#### Creating Requested Team & Team Dataframe

In [0]:
df = spark.sql("SELECT * FROM data")

In [0]:
team_object = (
    df.select("payload_pull_request.*")
    .distinct()
    .select(explode("requested_teams"))
    .select("col.*")
)

requested_teamDF = (
    df.select(
        col("payload_pull_request.id").alias("pull_request_id"),
        explode("payload_pull_request.requested_teams"),
    )
    .select("pull_request_id", col("col.id").alias("team_id"))
    .distinct()
)

In [0]:
teamDF = (
    team_object.select(
        col("id").cast(LongType()),
        "description",
        "html_url",
        "members_url",
        "name",
        "node_id",
        col("parent.id").cast(LongType()).alias("parent_id"),
        "permission",
        "privacy",
        "repositories_url",
        "slug",
        "url",
    )
    .distinct()
    .orderBy("id")
)

#### Writing Both to Database

In [0]:
requested_teamDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/requested-team")

In [0]:
teamDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/team")